In [ ]:
%%file keysched.m
function w = keysched(key)
    global S_box;
    global m;
    global polyaes;

    subkey = uint32(key);
    rcon = gf(1, m, polyaes);

    for r = 1:10
        % Rotword
        subword = subkey([2 3 4 1], end);
        % Subword
        subword = S_box(subword + 1);
        % Rcon
        subword(1) = bitxor(subword(1), rcon.x);
        w(:, 1, r) = bitxor(subkey(:, 1), subword);
        subword = w(:, 1, r);

        for wd = 2:4
            w(:, wd, r) = bitxor(subkey(:, wd), subword);
            subword = w(:, wd, r);
        end

        subkey = w(:, :, r);
        rcon = gf(2, m, polyaes) * rcon;
    end

end



In [2]:
%%file gen_s_box.m

function s_box = gen_s_box
    global m;
    global polyaes;
    Ta = gf([[1 0 0 0 1 1 1 1];
                 [1 1 0 0 0 1 1 1];
                 [1 1 1 0 0 0 1 1];
                 [1 1 1 1 0 0 0 1];
                 [1 1 1 1 1 0 0 0];
                 [0 1 1 1 1 1 0 0];
                 [0 0 1 1 1 1 1 0];
                 [0 0 0 1 1 1 1 1]], 1);
    % Calcul de l'inverse
    %s_box = gf((1:2^m-1),m,polyaes).^(-1);
    s_box = gf(ones(1, 2 ^ m - 1), m, polyaes) ./ gf((1:2 ^ m - 1), m, polyaes);
    s_box = [gf(0, m, polyaes) s_box];
    % Calcul de la fonction affine
    s_box = Ta * gf(de2bi(s_box.x', 'right-msb')', 1) + gf([1; 1; 0; 0; 0; 1; 1; 0], 1) * ones(1, 2 ^ m);
    s_box = bi2de(s_box.x', 'right-msb');
end


Created file 'c:\Users\Mathis\Desktop\Projet EMA\gen_s_box.m'.
